In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
! pip install streamlit

In [3]:
%%writefile app.py
import pickle
import keras
import keras.backend as K
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import streamlit as st
import re
from keras_preprocessing.sequence import pad_sequences
df = pd.read_csv("/content/drive/MyDrive/Digified/final.csv")
df.drop(['index','Unnamed: 0'], axis=1, inplace=True)
df.drop_duplicates(inplace = True)
tags = df['class']
encoder = LabelEncoder()
encoder.fit(tags)
def f1_metric(y_true, y_pred):
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  recall = true_positives / (possible_positives + K.epsilon())
  f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
  return f1_val
dependencies = {
    'f1_metric': f1_metric
}

model = keras.models.load_model('/content/drive/MyDrive/Digified/my_model.h5', custom_objects=dependencies)
# loading
with open('/content/drive/MyDrive/Digified/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

def prediction(name):
  tokens = tokenizer.texts_to_matrix([name], mode='tfidf')
  predict_x=model.predict(np.array(tokens)) 
  c=np.argmax(predict_x,axis=1)
  xc = encoder.inverse_transform(c)
  if predict_x[0][1]> 0.50 :
    if predict_x[0][1]- predict_x[0][0]>0.001:
      Confidence="True Name with High Confidence"
    else:
      Confidence="True Name with Low Confidence"
  else:
    Confidence="False Name"
  return xc[0],Confidence

st.set_page_config(page_title='real and fake names',page_icon=":tada:")
st.title("REAL and FAKE NAMES WEB APP")
name=st.text_input("Name")
test="not find"
Confidence="not find"
if st.button('Result'):
  test,Confidence=prediction(name)

st.title("first mode")
st.success(test)
st.success(Confidence)






Writing app.py


In [4]:
!streamlit run app.py & npx localtunnel --port 8501

[..................] / fetchMetadata: sill resolveWithNewModule yargs@17.1.1 ch

npx: installed 22 in 6.4s
your url is: https://sweet-cases-check-34-125-140-190.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.140.190:8501

2022-12-09 19:42:15.315481: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
1/1 [==============================] - 0s 213ms/step
  Stopping...
^C
